In [1]:
from astropy.io import ascii
from astropy.table import Table

In [44]:
data_start = 3
gaia = ascii.read('temperatures/gaia_temps.tsv', names='Target,Gmag,Teff_Gaia,Teff_Gaia_lower,Teff_Gaia_upper'.split(','), 
                  data_start=data_start, delimiter='\t')
spocs2005 = ascii.read('temperatures/spocs2005_temps.tsv', names='Target, Teff_spocs2005'.split(', '), 
                       data_start=data_start, delimiter='\t')
spocs2016 = ascii.read('temperatures/spocs2016_temps.tsv', names='Target, Teff_spocs2016'.split(', '), 
                       data_start=data_start, delimiter='\t')

In [45]:
from astropy.table import join

joined_temps = join(join(gaia, spocs2005, keys=['Target'], join_type='outer'), 
                    spocs2016, keys=['Target'], join_type='outer')

In [46]:
joined_temps.remove_column('Gmag')

In [47]:
import numpy as np
gaia_teffs = [joined_temps['Teff_Gaia'], 
              joined_temps['Teff_Gaia']-joined_temps['Teff_Gaia_lower'], 
              joined_temps['Teff_Gaia_upper']-joined_temps['Teff_Gaia']]

gaia_formatted = []
for mid, lower, upper in Table(gaia_teffs, names=['Mid', 'Lower', 'Upper']):
    if isinstance(mid, float):
        gaia_formatted.append('${0:.0f}^{{+{1:.0f}}}_{{-{2:.0f}}}$'.format(mid, upper, lower))
    else: 
        gaia_formatted.append('')

In [48]:
from astropy.table import Column
joined_temps.remove_columns(['Teff_Gaia', 'Teff_Gaia_lower', 'Teff_Gaia_upper'])
joined_temps.add_column(Column(gaia_formatted, 'T_eff_Gaia'))

In [49]:
joined_temps.sort("Target")

In [50]:
import sys
joined_temps.write(sys.stdout, format='latex')

\begin{table}
\begin{tabular}{cccc}
Target & Teff_spocs2005 & Teff_spocs2016 & T_eff_Gaia \\
51 Peg & 5787 & 5758 & $5830^{+214}_{-55}$ \\
EK Dra & 5845 & 5647 & $5584^{+115}_{-193}$ \\
EQ Vir &  &  & $4433^{+82}_{-106}$ \\
GJ702B &  & 5307 & $5305^{+341}_{-245}$ \\
GJ705 &  &  & $4465^{+102}_{-66}$ \\
GJ9781A &  &  & $4484^{+95}_{-110}$ \\
HAT-P-11 &  &  & $4757^{+154}_{-111}$ \\
HD10697 & 5680 & 5600 & $5654^{+36}_{-56}$ \\
HD110833 &  & 4999 & $5025^{+50}_{-82}$ \\
HD113827 &  &  & $4610^{+459}_{-191}$ \\
HD122120 &  &  & $4469^{+253}_{-144}$ \\
HD127506 &  &  & $4711^{+55}_{-83}$ \\
HD129333 & 5845 & 5647 & $5584^{+115}_{-193}$ \\
HD134319 & 5662 & 5733 & $5636^{+100}_{-54}$ \\
HD14039 &  &  & $5072^{+98}_{-73}$ \\
HD145675 & 5388 & 5280 & $5282^{+251}_{-73}$ \\
HD145742 &  &  & $4851^{+13}_{-75}$ \\
HD148467 &  &  & $4474^{+153}_{-201}$ \\
HD149957 &  &  & $4485^{+37}_{-89}$ \\
HD151288 &  &  & $4224^{+300}_{-270}$ \\
HD175742 &  & 4937 & $4874^{+43}_{-74}$ \\
HD178126 &  &  & $46

In [51]:
sinds = Table.read('/Users/bmmorris/git/arces_hk/freckles.tex')
sinds.sort('Target')

In [52]:
from astropy.table import hstack

full_table = join(joined_temps, sinds, keys='Target')

In [55]:
full_table.sort('$\log R^\prime_\mathrm{HK}$')

In [56]:
full_table.write(sys.stdout, format='latex')

\begin{table}
\begin{tabular}{cccccccc}
Target & Teff_spocs2005 & Teff_spocs2016 & T_eff_Gaia & Sp. Type & $S$ & $\log R^\prime_\mathrm{HK}$ & Active? \\
HD5857 &  &  & $4712^{+67}_{-71}$ & G5 & $0.11 \pm 0.01$ & -5.53 & Inactive \\
GJ705 &  &  & $4465^{+102}_{-66}$ & K2 & $0.12 \pm 0.01$ & -5.44 & Inactive \\
HD6497 &  &  & $4514^{+82}_{-71}$ & K2III-IV & $0.11 \pm 0.01$ & -5.41 & Inactive \\
HD89744 & 6291 &  & $6239^{+23}_{-84}$ & F7V & $0.14 \pm 0.01$ & -5.11 & Inactive \\
HD145742 &  &  & $4851^{+13}_{-75}$ & K0 & $0.15 \pm 0.01$ & -5.09 & Inactive \\
HD10697 & 5680 & 5600 & $5654^{+36}_{-56}$ & G3Va & $0.14 \pm 0.01$ & -5.01 & Inactive \\
HD221639 &  &  & $5102^{+139}_{-106}$ & K1V & $0.15 \pm 0.01$ & -4.96 & Inactive \\
HD210277 & 5555 & 5484 & $5535^{+42}_{-48}$ & G8V & $0.15 \pm 0.01$ & -4.94 & Inactive \\
HD145675 & 5388 & 5280 & $5282^{+251}_{-73}$ & K0V & $0.16 \pm 0.01$ & -4.94 & Inactive \\
HD86728 & 5700 & 5742 & $5832^{+251}_{-207}$ & G3VaHdel1 & $0.15 \pm 0.01$ & -4.93